**Lilian JAOUANNE**

**Bastian GARÇON**

Mecatro 3A FISA Promo 2026

# Base de Données sur la Qualité des Vins de Vinho Verde

## Origine des données

Les données sont extraites de l'[UCI Machine Learning Repository](https://archive.ics.uci.edu) et se rapportent spécifiquement aux variantes rouge et blanche des vins de Vinho Verde, une région notable du Portugal.

## Composition de la Base de Données

La base de données se divise en deux ensembles distincts, l'un pour les vins rouges et l'autre pour les vins blancs. Chaque ensemble comprend plusieurs variables qui mesurent les caractéristiques physicochimiques des vins ainsi que leur qualité sensorielle évaluée :
- Fixed Acidity: Acidité totale du vin, incluant les acides fixes et volatils.
- Volatile Acidity: Concentration en acide acétique, influant sur le goût vinaigré du vin.
- Citric Acid: Acidité apportée par l'ajout d'acide citrique, ajustant l'acidité des vins.
- Residual Sugar: Quantité de sucre restant post-fermentation.
- Chlorides: Teneur en chlorures, ou sel, du vin.
- Free Sulfur Dioxide: Quantité de dioxyde de soufre libre, prévenant l'oxydation et la croissance microbienne.
- Total Sulfur Dioxide: Totalité du dioxyde de soufre présent dans le vin.
- Density: Densité du vin, souvent liée à sa teneur en alcool.
- pH: Indice de l'acidité ou basicité du vin.
- Sulphates: Taux de sulfates ajoutés, augmentant le SO2 et aidant à la conservation du vin.
- Alcohol: Pourcentage d'alcool contenu dans le vin.
- Quality: Note de qualité attribuée par des dégustateurs, sur une échelle de 0 à 10.

## Problématiques Principales

### 1. Identification des Variables Influant sur la Densité

- **Objectif** : Détecter les variables physicochimiques qui influencent significativement la densité du vin, pour les vins rouges et blancs.
- **Approche** : Sélection de variables pertinentes à travers des techniques statistiques pour minimiser le bruit et se concentrer sur les contributeurs clés à la densité du vin.

### 2. Comparaison des Distributions de Variables Spécifiques

- **Objectif** : Examiner comment les distributions des variables telles que le pH, la densité, et l'acidité volatile varient entre les vins rouges et blancs.
- **Approche** : Utilisation de tests de normalité et d'analyse de variance (ANOVA) pour évaluer les différences statistiques entre les deux types de vin.

### 3. Modélisation de la Densité du Vin

- **Objectif** : Construire un modèle de régression linéaire multiple pour prédire la densité du vin à partir des variables sélectionnées, et comparer les modèles pour les vins rouges contre les vins blancs.
- **Approche** : Développement et validation d'un modèle de régression, évaluation de la précision et des performances du modèle, et analyse des interactions entre les variables.

### 4. Analyse des Associations entre Qualité du Vin et Autres Variables Catégorielles

- **Objectif** : Investiguer s'il existe des associations significatives entre les catégories de qualité du vin et d'autres variables catégorielles.
- **Approche** : Utilisation de tests statistiques pour évaluer l'association entre variables catégorielles et la qualité perçue du vin.

# Mise en pratique

## Librairies à importer

Avant toute chose, nous devons importer ces librairies :
- _Pandas_ : pour la partie dataframes
- _numpy_ : pour les tableaux
- _pyplot_ de _matplotlib_ : pour une visualisation graphique
- _seaborn_ : pour la visualisation de données
- _statmodels.api_ : pour le test ANOVA
- _shapiro_ de _scipy.stats_ : pour le test Shapiro puis _scipy.stats_ pour les fonctions statistiques.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import shapiro
import scipy.stats as stats

## Importation des bases de données

On importe les deux base de données grâce à _pandas_

In [ ]:
num_data_red = pd.read_csv("Bases de données/winequality-red.csv", sep=';', decimal='.')
num_data_white = pd.read_csv("Bases de données/winequality-white.csv", sep=';', decimal='.')

## Table de corrélation

On commence faire une table de corrélation pour avoir une idée des variables qui influence la densité. Pour cela nous avons crée une fonction pemettant d'afficher une heat map à partir d'une matrice de corrélation.

In [ ]:
def heatMap(M, title:str, center=0, vmax=1, vmin=-1):
    plt.title(f'Table de corrélation {title}')
    # Tracer la heatmap avec la colormap personnalisée
    sns.heatmap(M, annot=False, cmap='coolwarm', center=center, vmax=vmax, vmin=vmin)
    
    # Ajouter les annotations pour chaque cellule de la heatmap
    for i in range(len(M)):
        for j in range(len(M)):
            plt.text(j + 0.5, i + 0.5, '{:.2f}'.format(M.iloc[i, j]),
                     ha='center', va='center', color='black', fontsize=9)
    plt.show()

Ainsi pour le vin rouge :

In [ ]:
M_corr_red = num_data_red.corr()
heatMap(M_corr_red, 'Vin rouge')

Et le vin blanc :

In [ ]:
M_corr_white = num_data_white.corr()
heatMap(M_corr_white, 'Vin blanc')